In [1]:
!pip install timm
!pip install einops
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from PIL import Image
from einops import rearrange
import torchvision
from torch import nn, optim
import timm
import imageio.v2 as imageio
from torchvision.transforms import v2
from glob import glob
from sklearn.preprocessing import LabelEncoder
import time
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

from torch import linalg as LA
import prettytable
import time, random,timeit
import sys
sys.setrecursionlimit(15000)

from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
import seaborn as sns

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.8 MB/s eta 0:00:00


In [2]:
normalize = transforms.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5])
Resize = transforms.Resize((384,384))

pin_memory = True
nw = 6 

data_transform = {
        "train": transforms.Compose([Resize,
                                     transforms.RandomVerticalFlip(),
                                     transforms.ToTensor(),
                                     normalize]),
        "val": transforms.Compose([Resize,
                                    transforms.ToTensor(),
                                    normalize]),
        "test": transforms.Compose([Resize,
                                    transforms.ToTensor(),
                                    normalize]),}    

train_dataset = datasets.ImageFolder(root='/kaggle/input/ham10000-augmented/ham10000_augmented/train525e384',transform=data_transform["train"])
val_dataset = datasets.ImageFolder(root='/kaggle/input/ham10000-augmented/ham10000_augmented/val525e384png',transform=data_transform["val"])
test_dataset = datasets.ImageFolder(root='/kaggle/input/ham10000-augmented/ham10000_augmented/test525png384',transform=data_transform["test"])

train_num = len(train_dataset)
val_num = len(val_dataset)
test_num = len(test_dataset)

train_loader = DataLoader(train_dataset,batch_size=168,
                                                   pin_memory=pin_memory,
                                                   shuffle=True,num_workers=nw)
val_loader = DataLoader(val_dataset,batch_size=21,
                                                   pin_memory=pin_memory,
                                                   shuffle=True,num_workers=nw)
test_loader = DataLoader(test_dataset,batch_size=21,
                                                  pin_memory=pin_memory,
                                                  shuffle=False,num_workers=nw)

print("using {} images for training, {} images for val, {} images for testing.".format(train_num,val_num,test_num))

    
data_list = test_dataset.class_to_idx
cla_dict = dict((val, key) for key, val in data_list.items())
n_classes  = len(data_list)

using 51646 images for training, 1006 images for val, 828 images for testing.


/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [3]:
train_dataset.class_to_idx

{'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv': 5, 'vasc': 6}

In [4]:
loc_time = time.strftime("%H%M%S", time.localtime()) 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ratio = 8
# c_outs= 128
reduction = 1 # default:1
n_iter = 0

def squash(inputs,ep_iter=n_iter,total=100):
    beta = 1.45 - 0.22* (ep_iter/total)
    mag_sq = torch.sum(inputs**2, dim=2, keepdim=True)
    mag = torch.sqrt(mag_sq)
    s = (mag_sq / (beta + mag_sq)) * (inputs / mag)
    return s


class CapsNet(nn.Module):
    def __init__(self,conv_inputs,
                 num_classes=7,
                 init_weights=False,
                 conv_outputs = 128,
                 primary_units = 8,#8,
                 primary_unit_size = 576,# 16 * 6 * 6,
                 output_unit_size = 16,):
        super().__init__()
        
        self.Convolution = nn.Sequential(nn.Conv2d(conv_inputs, conv_outputs, 21,stride=2),
                                        nn.BatchNorm2d(conv_outputs),
                                        nn.ReLU(inplace=True),)

        

        # self.Pool = nn.FractionalMaxPool2d(3, output_size=(20))
        self.Pool = nn.AdaptiveMaxPool2d(20)
        #Attention
        self.CBAM = Conv_CBAM(conv_outputs,conv_outputs)
        #Capsule
        self.primary = Primary_Caps(in_channels=conv_outputs,#128
                                    caps_units=primary_units,#8
                                    )

        self.digits = Digits_Caps(in_units=primary_units,#8
                                   in_channels=primary_unit_size,#16*6*6=576
                                   num_units=num_classes,#classification_num
                                   unit_size=output_unit_size,#16
                                   )
        if init_weights:
            self._initialize_weights()
        
    def forward(self, x):
        x = self.Convolution(x)
        x = self.Pool(x)      
        x = self.CBAM(x)
        out = self.digits(self.primary(x))
        return out

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):# or isinstance(m, nn.BatchNorm2d):
                nn.init.normal_(m.weight, 0, 0.01)
                # nn.init.constant_(m.bias, 0)
    #margin_loss           
    def loss(self,img_input, target, epoch=1,epoch_total=100, size_average=True):
    # def loss(self,img_input, target, size_average=True):
        batch_size = img_input.size(0)
        # coefficient = epoch/epoch_total
        
        v_mag = LA.norm(img_input,ord=2,dim=(2,3),keepdim=True) #largest singular value
        zero = Variable(torch.zeros(1)).to(device)
        m_plus  = 0.9 #- (coefficient/10) 
        m_minus =0.1 #+ (coefficient/10)
        max_l = torch.max(m_plus - v_mag, zero).view(batch_size, -1)**2
        max_r = torch.max(v_mag - m_minus, zero).view(batch_size, -1)**2
        
        # init_lambda = 0.5 
        loss_lambda = 0.5 #+ coefficient
        # print(f"lambda:{loss_lambda}")
        T_c = target
        L_c = T_c * max_l + loss_lambda * (1.0 - T_c) * max_r
        L_c = torch.sum(L_c,1)
        
        if size_average:
            L_c = torch.mean(L_c)

        return L_c

    def update_n_iter(self, ep_iter):
        if ep_iter > 100 and ep_iter % 10 == 0:
            ep_iter -=100
            self.primary.n_iter = ep_iter
            self.digits.n_iter = ep_iter
            beta = 1.45 - 0.22* (ep_iter/100)
            print(f"beta:{beta}")
        
        
class Primary_Caps(nn.Module):
    def __init__(self, in_channels, caps_units):
        super(Primary_Caps, self).__init__()
        self.n_iter = n_iter
        self.in_channels = in_channels
        self.caps_units = caps_units
        
        def create_conv_unit(unit_idx):
            unit = ConvUnit(in_channels=in_channels)
            self.add_module("Caps_" + str(unit_idx), unit)
            return unit
        self.units = [create_conv_unit(i) for i in range(self.caps_units)]
   
    #no_routing
    def forward(self, x):
        # Get output for each unit.
        # Each will be (batch, channels, height, width).
        u = [self.units[i](x) for i in range(self.caps_units)]
        # Stack all unit outputs (batch, unit, channels, height, width).
        u = torch.stack(u, dim=1)
        # Flatten to (batch, unit, output).
        u = u.view(x.size(0), self.caps_units, -1)

        return squash(u,self.n_iter)
    
class Digits_Caps(nn.Module):
    def __init__(self, in_units, in_channels, num_units, unit_size):
        super(Digits_Caps, self).__init__()
        self.n_iter = n_iter
        self.in_units = in_units
        self.in_channels = in_channels
        self.num_units = num_units
        
        self.W = nn.Parameter(torch.randn(1, in_channels, self.num_units, unit_size, in_units))
        # self.w = [1,576,7,16,8]
        
    #routing
    def forward(self, x):
        batch_size = x.size(0)    
        # (batch, in_units, features) -> (batch, features, in_units)
        x = x.transpose(1, 2)        
        # (batch, features, in_units) -> (batch, features, num_units, in_units, 1)
        x = torch.stack([x] * self.num_units, dim=2).unsqueeze(4)        
        # (batch, features, in_units, unit_size, num_units)
        W = torch.cat([self.W] * batch_size, dim=0)
        # Transform inputs by weight matrix.
        # (batch_size, features, num_units, unit_size, 1)
        u_hat = torch.matmul(W, x)
        # Initialize routing logits to zero.
        b_ij = Variable(torch.zeros(1, self.in_channels, self.num_units, 1)).to(device)
        
        num_iterations = 3
        for iteration in range(num_iterations):
            # Convert routing logits to softmax.
            # (batch, features, num_units, 1, 1)
            #c_ij = F.softmax(b_ij, dim=0)
            c_ij = b_ij.softmax(dim=1)
            c_ij = torch.cat([c_ij] * batch_size, dim=0).unsqueeze(4)

            # Apply routing (c_ij) to weighted inputs (u_hat).
            # (batch_size, 1, num_units, unit_size, 1)
            s_j = torch.sum(c_ij * u_hat, dim=1, keepdim=True)

            # (batch_size, 1, num_units, unit_size, 1)
            v_j = squash(s_j,self.n_iter)

            # (batch_size, features, num_units, unit_size, 1)
            v_j1 = torch.cat([v_j] * self.in_channels, dim=1)

            # (batch_size, features, num_units, 1)
            u_vj1 = torch.matmul(u_hat.transpose(3, 4), v_j1).squeeze(4).mean(dim=0, keepdim=True)

            # Update b_ij (routing)
            b_ij = u_vj1

        # return v_j.squeeze(1)
        return rearrange(v_j.squeeze(1), 'b c (g h) w -> b c g (h w)',g=4)
                
class ConvUnit(nn.Module):
    def __init__(self, in_channels):
        super(ConvUnit, self).__init__()
        Caps_out = in_channels // ratio# 16
        self.Cpas = nn.Sequential(
                        # nn.Conv2d(in_channels,Caps_out,(9,9),stride=2,groups=Caps_out, bias=False),
                        nn.Conv2d(in_channels,in_channels,(9,1),stride=1, bias=False),
                        nn.Conv2d(in_channels,Caps_out,(1,9),stride=2,groups=Caps_out, bias=False),
                    )

    def forward(self, x):
        output = self.Cpas(x)
        return output

class Conv_CBAM(nn.Module):
    def __init__(self, c1, c2, k=1, s=1, p=None, g=1, act=True):
        super(Conv_CBAM, self).__init__()
        self.conv = nn.Conv2d(c1, c2, k, s, autopad(k, p), groups=g, bias=False)
        self.bn = nn.BatchNorm2d(c2)
        self.act = nn.Hardswish() if act else nn.Identity()
        self.ca = ChannelAttention(c2, reduction=reduction)
        self.sa = SpatialAttention()

    def forward(self, x):
        x = self.act(self.bn(self.conv(x)))
        x = self.ca(x) * x
        x = self.sa(x) * x
        return x
    
def autopad(k, p=None):  # kernel, padding
    if p is None:
        p = k // 2 if isinstance(k, int) else [x // 2 for x in k]
    return p

# SAM
class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=3): # default:3
        super(SpatialAttention, self).__init__()

        assert kernel_size in (3, 7)
        padding = 3 if kernel_size == 7 else 1

        self.conv1 = nn.Conv2d(2, 1, kernel_size,padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # This is different from the paper[S. Woo, et al. "CBAM: Convolutional Block Attention Module,"].
        avg_out = torch.mean(x, dim=1, keepdim=True)#The different channels are averaged and converted to 1 channel.
        max_out, _ = torch.max(x, dim=1, keepdim=True)#Maximizing the different channels and making them 1-channel.
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

# CAM
class ChannelAttention(nn.Module):
    def __init__(self, channels, reduction=16):
        super(ChannelAttention, self).__init__()
        me_c = channels // reduction
        self.avg_pool = nn.AdaptiveAvgPool2d(1) 
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc1   = nn.Conv2d(channels, me_c, 1, bias=False)
        self.relu1 = nn.ReLU(inplace=True)
        self.fc2   = nn.Conv2d(me_c, channels, 1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)

In [5]:
n_channels = 3 #RGB

network = CapsNet(conv_inputs=n_channels, 
                     num_classes=7,# category_number
                     init_weights=True,)
network = network.to(device)

In [6]:
network.load_state_dict(torch.load('/kaggle/input/capsnet_78/pytorch/capsnet_78/2/best_capsnet.pth',
                                  map_location=torch.device('cpu')))

<All keys matched successfully>

In [7]:
from tqdm.notebook import tqdm
loss_log = []
acc_log = []
network.eval()

predictions = []
gt = []

def one_hot(x, length):
    batch_size = x.size(0)
    x_one_hot = torch.zeros(batch_size, length)
    for i in range(batch_size):
        x_one_hot[i, x[i]] = 1.0
    return x_one_hot

cor_loss,correct,Auc, Acc= 0, 0, 0, 0
evl_tmp_result = torch.zeros(n_classes,n_classes)
    

data_loader = test_loader
tmp_size = 21
data_num = test_num
        
steps_num = len(data_loader)
    
with torch.no_grad():
    for batch_idx, (data, target) in enumerate(data_loader):
        batch_idx +=1
        target_indices = target#torch.Size([batch, 7])  
        target_one_hot = one_hot(target, length=n_classes)            
        data, target = Variable(data).to(device), Variable(target_one_hot).to(device)

        output= network(data)#torch.Size([batch_size, 7, 16, 1])         
        v_mag = LA.norm(output,ord='nuc',dim=(2,3), keepdim=True)#
        pred = v_mag.data.max(1, keepdim=True)[1].cpu()#[9, 2, 1, 1, 6,..., 1, 4, 6, 5, 7,]
            
        if batch_idx % steps_num == 0 and data_num % tmp_size != 0:
            tmp_size = data_num % tmp_size
                          
        for i in range(tmp_size):
            pred_y = pred.numpy()
            evl_tmp_result[target_indices[i]][pred_y[i]] +=1 
            predictions.append(pred_y[i])
            gt.append(target_indices[i])

    diag_sum = torch.sum(evl_tmp_result.diagonal())
    all_sum = torch.sum(evl_tmp_result) 
    test_acc = 100. * float(torch.div(diag_sum,all_sum)) 

In [8]:
test_acc

94.20289993286133

In [13]:
np.squeeze(gt)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,

In [15]:
from sklearn.metrics import classification_report

print(classification_report(np.squeeze(gt), np.squeeze(predictions)))

              precision    recall  f1-score   support

           0       0.89      0.35      0.50        23
           1       0.76      0.96      0.85        26
           2       0.77      0.82      0.79        66
           3       0.50      0.17      0.25         6
           4       0.91      0.62      0.74        34
           5       0.97      1.00      0.98       663
           6       1.00      0.90      0.95        10

    accuracy                           0.94       828
   macro avg       0.83      0.69      0.72       828
weighted avg       0.94      0.94      0.94       828



In [17]:
from sklearn.metrics import confusion_matrix

confusion_matrix(np.squeeze(gt), np.squeeze(predictions))

array([[  8,   6,   8,   0,   0,   1,   0],
       [  0,  25,   1,   0,   0,   0,   0],
       [  0,   1,  54,   1,   1,   9,   0],
       [  0,   0,   3,   1,   0,   2,   0],
       [  1,   1,   4,   0,  21,   7,   0],
       [  0,   0,   0,   0,   1, 662,   0],
       [  0,   0,   0,   0,   0,   1,   9]])

In [24]:
def test(model, loader):
    loss_log = []
    acc_log = []
    model.eval()
    
    startT = timeit.default_timer() 
    for data, target in loader:
        data = data.to(device)
        target = target.to(device)

        with torch.no_grad():
            predictions = model(data)
            loss = nn.functional.cross_entropy(predictions, target)

            loss_log.append(loss.item())

            acc = sum(torch.argmax(predictions.cpu(), dim=1) ==
                   target.cpu()) / target.cpu().shape[0]

            acc_log.append(acc.item())
            
    endT = timeit.default_timer()
    run_time = endT-startT

    return np.mean(loss_log), np.mean(acc_log)

def train_epoch(model, optimizer, train_loader):
    loss_log = []
    acc_log = []
    model.train()

    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)

        optimizer.zero_grad()
        predictions = model(data)
        loss = nn.functional.cross_entropy(predictions, target)
        loss.backward()
        optimizer.step()

        loss_log.append(loss.item())

        with torch.no_grad():
            acc = sum(torch.argmax(predictions.cpu(), dim=1) ==
                   target.cpu()) / target.cpu().shape[0]

        acc_log.append(acc.item())

    return loss_log, acc_log

def train(model, optimizer, n_epochs, train_loader, val_loader, scheduler=None):
    train_loss_log, train_acc_log, val_loss_log, val_acc_log = [], [], [], []
    

    for epoch in range(n_epochs):
        startT = timeit.default_timer() 
        train_loss, train_acc = train_epoch(model, optimizer, train_loader)
        val_loss, val_acc = test(model, val_loader)

        train_loss_log.extend(train_loss)
        train_acc_log.extend(train_acc)

        val_loss_log.append(val_loss)
        val_acc_log.append(val_acc)

        print(f"Epoch {epoch}")
        print(f" train loss: {np.mean(train_loss)}, train acc: {np.mean(train_acc)}")
        print(f" val loss: {val_loss}, val acc: {val_acc}\n")
        endT = timeit.default_timer()
        run_time = endT-startT
        print("Running:[{:.2f}s]".format(run_time))

        if scheduler is not None:
            scheduler.step(val_acc)
            
    return train_loss_log, train_acc_log, val_loss_log, val_acc_log

In [19]:
normalize = transforms.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5])
Resize = transforms.Resize((384,384))

pin_memory = True
nw = 2

data_transform = {
        "train": transforms.Compose([Resize,
                                     transforms.RandomVerticalFlip(),
                                     transforms.ToTensor(),
                                     normalize]),
        "val": transforms.Compose([Resize,
                                    transforms.ToTensor(),
                                    normalize]),
        "test": transforms.Compose([Resize,
                                    transforms.ToTensor(),
                                    normalize]),}    

train_dataset = datasets.ImageFolder(root='/kaggle/input/ham10000-augmented/ham10000_augmented/train525e384',transform=data_transform["train"])
val_dataset = datasets.ImageFolder(root='/kaggle/input/ham10000-augmented/ham10000_augmented/val525e384png',transform=data_transform["val"])
test_dataset = datasets.ImageFolder(root='/kaggle/input/ham10000-augmented/ham10000_augmented/test525png384',transform=data_transform["test"])

train_num = len(train_dataset)
val_num = len(val_dataset)
test_num = len(test_dataset)

train_loader = DataLoader(train_dataset,batch_size=32,
                                                   pin_memory=pin_memory,
                                                   shuffle=True,num_workers=nw)
val_loader = DataLoader(val_dataset,batch_size=21,
                                                   pin_memory=pin_memory,
                                                   shuffle=True,num_workers=nw)
test_loader = DataLoader(test_dataset,batch_size=21,
                                                  pin_memory=pin_memory,
                                                  shuffle=False,num_workers=nw)

print("using {} images for training, {} images for val, {} images for testing.".format(train_num,val_num,test_num))

    
data_list = test_dataset.class_to_idx
cla_dict = dict((val, key) for key, val in data_list.items())
n_classes  = len(data_list)

using 51646 images for training, 1006 images for val, 828 images for testing.


In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = timm.create_model('resnet50', pretrained=True)
num_classes = 7

num_features = model.fc.in_features

model.fc = nn.Sequential(
    nn.Linear(num_features, num_classes)
)
model=model.to(device)

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

In [ ]:
optimizer = optim.Adam(model.parameters())
train_loss_log, train_acc_log, val_loss_log, val_acc_log = train(model, optimizer,
                                                                 75, train_loader, val_loader)

Epoch 0
 train loss: 0.2266153531700668, train acc: 0.9179639095415117
 val loss: 0.40779986915489036, val acc: 0.8561508022248745

Running:[1384.42s]
Epoch 1
 train loss: 0.14952602002958973, train acc: 0.9454744940132014
 val loss: 0.6216749070833126, val acc: 0.8281641701857249

Running:[1382.47s]
Epoch 2
 train loss: 0.11298929677169442, train acc: 0.9602101404206017
 val loss: 0.4559406014159322, val acc: 0.8667502167324225

Running:[1382.17s]
Epoch 3
 train loss: 0.09307676093633226, train acc: 0.9670823007046512
 val loss: 0.3875345954826723, val acc: 0.8689431995153427

Running:[1382.13s]
Epoch 4
 train loss: 0.07798743656247305, train acc: 0.9730096034696406
 val loss: 0.5003325294528622, val acc: 0.8745822968582312

Running:[1382.21s]
Epoch 5
 train loss: 0.06345884005001473, train acc: 0.9783534696406444
 val loss: 0.37894320910951745, val acc: 0.8757832149664561

Running:[1382.22s]
Epoch 6
 train loss: 0.06197867174604176, train acc: 0.9786413155745396
 val loss: 0.60408879

In [26]:
test(model, test_loader)

(0.7355437499355049, 0.8984127018600703)

In [27]:
torch.save(model.state_dict(), 'gpu_11_epochs_resnet.pth')

In [35]:
preds = []
gt = []

loss_log = []
acc_log = []
model.eval()
    
startT = timeit.default_timer() 
for data, target in test_loader:
    data = data.to(device)
    target = target.to(device)

    with torch.no_grad():
        predictions = model(data)
        loss = nn.functional.cross_entropy(predictions, target)

        loss_log.append(loss.item())

        acc = sum(torch.argmax(predictions.cpu(), dim=1) ==
                target.cpu()) / target.cpu().shape[0]

        acc_log.append(acc.item())
        
        for i in range(len(target)):
            preds.append(torch.argmax(predictions[i].cpu()))
            gt.append(target[i].cpu())
            
endT = timeit.default_timer()
run_time = endT-startT

In [37]:
print(classification_report(np.squeeze(gt), np.squeeze(preds)))

              precision    recall  f1-score   support

           0       0.57      0.57      0.57        23
           1       0.79      0.85      0.81        26
           2       0.83      0.44      0.57        66
           3       1.00      0.33      0.50         6
           4       0.55      0.35      0.43        34
           5       0.93      0.99      0.96       663
           6       1.00      0.90      0.95        10

    accuracy                           0.90       828
   macro avg       0.81      0.63      0.68       828
weighted avg       0.89      0.90      0.89       828

